In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bash-8-0-round-2/train.csv
/kaggle/input/bash-8-0-round-2/test.csv


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [3]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/bash-8-0-round-2/train.csv").fillna("")
test_df  = pd.read_csv("/kaggle/input/bash-8-0-round-2/test.csv").fillna("")


print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)


Train shape: (20624, 10)
Test shape : (5157, 9)


In [4]:
import os

for root, dirs, files in os.walk("/kaggle//input"):
    for file in files:
        print(os.path.join(root, file))
train_df.head()
train_df.columns

/kaggle//input/bash-8-0-round-2/train.csv
/kaggle//input/bash-8-0-round-2/test.csv


Index(['id', 'Headline', 'Importance Score', 'Lead Types', 'Power Mentions',
       'Agencies', 'Reasoning', 'Key Insights', 'Tags', 'Source File'],
      dtype='object')

In [5]:
text_cols = [
    "Headline",
    "Lead Types",
    "Power Mentions",
    "Agencies",
    "Reasoning",
    "Key Insights",
    "Tags"
]

for col in text_cols:
    train_df[col] = train_df[col].fillna("")
    test_df[col]  = test_df[col].fillna("")

In [6]:
train_df[text_cols].isna().sum()


Headline          0
Lead Types        0
Power Mentions    0
Agencies          0
Reasoning         0
Key Insights      0
Tags              0
dtype: int64

In [7]:
train_df["combined_text"] = train_df[text_cols].agg(" ".join, axis=1)
test_df["combined_text"]  = test_df[text_cols].agg(" ".join, axis=1)
train_df["combined_text"].head(1)


0    Historical analysis of Arab social cohesion an...
Name: combined_text, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=40000,
    ngram_range=(1, 2),
    min_df=2,
    stop_words="english"
)

X_train = tfidf.fit_transform(train_df["combined_text"])
X_test  = tfidf.transform(test_df["combined_text"])


In [9]:
from sklearn.model_selection import train_test_split

y = train_df["Importance Score"]

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y,
    test_size=0.2,
    random_state=42
)


In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_tr, y_tr)

rf_preds = rf.predict(X_val)
rmse_rf = mean_squared_error(y_val, rf_preds)**0.5

print("RF RMSE:", rmse_rf)


RF RMSE: 5.75633486360735


In [30]:
"Importance Score" in train_df["combined_text"].iloc[0]


False

In [31]:
X_train = tfidf.fit_transform(train_df["combined_text"])
X_test  = tfidf.transform(test_df["combined_text"])


In [32]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmses = []

for train_idx, val_idx in kf.split(X_train):
    X_tr_k, X_val_k = X_train[train_idx], X_train[val_idx]
    y_tr_k, y_val_k = y.iloc[train_idx], y.iloc[val_idx]

    model = xgb.XGBRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",
        random_state=42
    )

    model.fit(X_tr_k, y_tr_k)
    preds = model.predict(X_val_k)
    rmses.append(mean_squared_error(y_val_k, preds, squared=False))

print("CV RMSEs:", rmses)
print("Mean CV RMSE:", np.mean(rmses))


NameError: name 'xgb' is not defined

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmses = []

for train_idx, val_idx in kf.split(X_train):
    X_tr_k, X_val_k = X_train[train_idx], X_train[val_idx]
    y_tr_k, y_val_k = y.iloc[train_idx], y.iloc[val_idx]

    model = xgb.XGBRegressor(
        n_estimators=800,
        max_depth=6,
        learning_rate=0.5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",
        random_state=42
    )

    model.fit(X_tr_k, y_tr_k)
    preds = model.predict(X_val_k)
    rmses.append(mean_squared_error(y_val_k, preds, squared=False))

print("CV RMSEs:", rmses)
print("Mean CV RMSE:", np.mean(rmses))


In [ ]:
submission = pd.DataFrame({'id': test['id'], 'Importance Score': np.clip(preds, 0, 100)})
submission.to_csv('/kaggle/working/submission.csv', index=False)
print("Saved 'submission.csv'")
submission.head()